<a href="https://colab.research.google.com/github/Tannybuoy/Tannybuoy.github.io/blob/master/Testing_Top_P%2C_Top_K_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this FIRST to fix widget metadata
import json

# Get notebook metadata
from google.colab import _message
notebook = _message.blocking_request('get_ipynb')

# Remove problematic widget metadata
if 'metadata' in notebook and 'widgets' in notebook['metadata']:
    del notebook['metadata']['widgets']

print("✓ Widget metadata cleared!")

In [ ]:
# Cell 1: Install dependencies (run once)
!pip install transformers torch accelerate -q



In [ ]:
# Cell 2: Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



In [ ]:
# Cell 3: Load model
print("Loading model...")
model_name = "gpt2"  # Small, fast model - good for learning
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
print("Model loaded!")



In [ ]:
# Add this line to fix the warning
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Cell 4: Generate with top-k
def generate_text(prompt, top_k=50, temperature=0.7, max_tokens=50):
    """Generate text with top-k sampling"""
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    output = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        do_sample=True,
        top_k=top_k,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)



In [ ]:
# Cell 5: Test it!
prompt = "The future of AI product management is"
result = generate_text(prompt, top_k=50, temperature=0.7)
print(result)



In [ ]:
# Cell 6: Compare different top-k values
print("Comparing different top-k values:\n")

for k in [10, 30, 50, 100]:
    print(f"--- Top-k = {k} ---")
    result = generate_text(prompt, top_k=k, temperature=0.7, max_tokens=30)
    print(result)
    print()

In [ ]:
# Test the same prompt multiple times with the same top-k
# to see randomness

prompt = "The future of AI product management is"

print("Running top-k=50 five times:\n")
for i in range(5):
    result = generate_text(prompt, top_k=50, temperature=0.7, max_tokens=30)
    print(f"{i+1}. {result}\n")

In [ ]:
# Conservative: narrow choices, low randomness
generate_text(prompt, top_k=20, temperature=0.5)

# Balanced: medium choices, medium randomness
generate_text(prompt, top_k=50, temperature=0.7)

# Wild: wide choices, high randomness
generate_text(prompt, top_k=100, temperature=1.0)

In [ ]:
# Cell 7: Top-P Generation Function
def generate_with_top_p(prompt, top_p=0.9, temperature=0.7, max_tokens=50):
    """
    Generate text with top-p (nucleus) sampling

    Args:
        prompt: Input text
        top_p: Cumulative probability threshold (0.0 to 1.0)
               Higher = more diverse, Lower = more focused
        temperature: Randomness control
        max_tokens: Number of tokens to generate
    """
    encoded = tokenizer(prompt, return_tensors="pt")

    output = model.generate(
        encoded['input_ids'],
        attention_mask=encoded['attention_mask'],
        max_new_tokens=max_tokens,
        do_sample=True,
        top_p=top_p,              # 🔑 This is the key parameter!
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)



In [ ]:
# Cell 8: Test it
result = generate_with_top_p(
    "The future of AI product management is",
    top_p=0.9,
    temperature=0.7
)
print(result)



In [ ]:
# Cell 9: Compare different top-p values
print("\n" + "="*60)
print("COMPARING DIFFERENT TOP-P VALUES")
print("="*60 + "\n")

prompt = "The future of AI product management is"

for p in [0.5, 0.7, 0.9, 0.95]:
    print(f"--- Top-p = {p} ---")
    result = generate_with_top_p(prompt, top_p=p, temperature=0.7, max_tokens=40)
    print(result)
    print()

In [ ]:
# Cell 8: Visual Comparison
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import matplotlib.pyplot as plt
import numpy as np

def visualize_top_p_vs_top_k(prompt, top_k=50, top_p=0.9):
    """Visualize how top-k and top-p filter differently"""

    # Get probabilities
    encoded = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model(encoded['input_ids'])
        logits = outputs.logits[0, -1, :]

    probs = torch.softmax(logits, dim=0).numpy()
    sorted_indices = np.argsort(probs)[::-1]
    sorted_probs = probs[sorted_indices]

    # Calculate cumulative for top-p
    cumsum = np.cumsum(sorted_probs)
    top_p_cutoff = np.searchsorted(cumsum, top_p) + 1

    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Top-K visualization
    x = range(1, 101)
    y = sorted_probs[:100]
    colors = ['steelblue' if i < top_k else 'lightgray' for i in range(100)]
    ax1.bar(x, y, color=colors, alpha=0.7)
    ax1.axvline(top_k, color='red', linestyle='--', linewidth=2, label=f'Top-K cutoff ({top_k})')
    ax1.set_title(f'Top-K = {top_k} (Fixed Number)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Token Rank')
    ax1.set_ylabel('Probability')
    ax1.legend()
    ax1.text(0.5, 0.95, f'Always uses {top_k} tokens',
             transform=ax1.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # Top-P visualization
    colors_p = ['green' if i < top_p_cutoff else 'lightgray' for i in range(100)]
    ax2.bar(x, y, color=colors_p, alpha=0.7)
    ax2.axvline(top_p_cutoff, color='red', linestyle='--', linewidth=2,
                label=f'Top-P cutoff ({top_p_cutoff} tokens for {top_p})')
    ax2.set_title(f'Top-P = {top_p} (Adaptive)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Token Rank')
    ax2.set_ylabel('Probability')
    ax2.legend()
    ax2.text(0.5, 0.95, f'Uses {top_p_cutoff} tokens ({cumsum[top_p_cutoff-1]:.1%} mass)',
             transform=ax2.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

    plt.tight_layout()
    plt.show()

    # Show actual tokens
    print(f"\nPrompt: '{prompt}'\n")
    print(f"Top-K ({top_k}) would include tokens 1-{top_k}")
    print(f"Top-P ({top_p}) includes tokens 1-{top_p_cutoff} (reaching {cumsum[top_p_cutoff-1]:.1%} probability)\n")

    print("Top 10 tokens:")
    for i in range(10):
        token = tokenizer.decode([sorted_indices[i]])
        in_k = "✓" if i < top_k else "✗"
        in_p = "✓" if i < top_p_cutoff else "✗"
        print(f"{i+1}. '{token}' - {sorted_probs[i]:.4f} | Top-K: {in_k} | Top-P: {in_p}")

# Run visualization
visualize_top_p_vs_top_k("The cat sat on the", top_k=50, top_p=0.9)

In [ ]:
# Cell 10: Interactive comparison tool
def compare_all_methods(prompt, num_samples=3):
    """Generate with different methods to see the differences"""

    print(f"Prompt: '{prompt}'\n")
    print("="*70)

    methods = [
        ("Greedy (no sampling)", {"do_sample": False}),
        ("Top-K only (k=50)", {"do_sample": True, "top_k": 50, "temperature": 0.7}),
        ("Top-P only (p=0.9)", {"do_sample": True, "top_p": 0.9, "temperature": 0.7}),
        ("Both (k=50, p=0.9)", {"do_sample": True, "top_k": 50, "top_p": 0.9, "temperature": 0.7}),
    ]

    for method_name, params in methods:
        print(f"\n📌 {method_name}")
        print("-" * 70)

        for i in range(num_samples):
            encoded = tokenizer(prompt, return_tensors="pt")
            output = model.generate(
                encoded['input_ids'],
                attention_mask=encoded['attention_mask'],
                max_new_tokens=30,
                pad_token_id=tokenizer.eos_token_id,
                **params
            )
            result = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"{i+1}. {result}")
        print()

# Run comparison
compare_all_methods("The hardest part of being a PM is", num_samples=2)

In [ ]:
# Install JupyterLite
!pip install jupyterlite-core

